In [1]:
import pandas as pd
import sqlite3
import logging
import os
from pathlib import Path
from dotenv import load_dotenv
import datetime



In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def get_recent_data(conn: sqlite3.Connection) -> tuple[pd.DataFrame, pd.DataFrame]:
   """Obtiene datos necesarios de la base de datos"""
   anio_actual = datetime.datetime.now().year
   periodo_inicial = int(f"{anio_actual - 1}00")

   base = pd.read_sql_query(f"SELECT * FROM base_balance_ultimos_periodos WHERE periodo > {periodo_inicial}", conn)
   filtro = pd.read_sql_query("SELECT * FROM conceptos_reportes WHERE es_subramo is FALSE", conn)
   params_full = pd.read_sql_query("SELECT * FROM parametros_reportes", conn)

   parametros_reportes = params_full.merge(filtro, on=['reporte','referencia'], how='inner')
   logging.info(f"Base generada con {len(base):,} registros")

   return base, parametros_reportes

def generate_subramos_table(data: pd.DataFrame, codigos: dict) -> pd.DataFrame:
   """Genera tabla agregada por diferentes conceptos que no tienen subramo asociado"""
   result = data.copy()

   for concepto, mapping in codigos.items():
       result[concepto] = result['cod_cuenta'].map(mapping) * result['importe']

   return result.groupby(
       by=['cod_cia', 'periodo'],
       as_index=False
   ).agg(
       resultado_tecnico=('resultado_tecnico', 'sum'),
       resultado_financiero=('resultado_financiero', 'sum'),
       resultado_operaciones=('resultado_operaciones', 'sum'),
       impuesto_ganancias=('impuesto_ganancias', 'sum'),
       deudas_con_asegurados=('deudas_con_asegurados', 'sum'),
       deudas_con_asegurados_ac_reaseguros=('deudas_con_asegurados_ac_reaseguros', 'sum'),
       disponibilidades=('disponibilidades', 'sum'),
       inmuebles_inversion=('inmuebles_inversion', 'sum'),
       inmuebles_uso_propio=('inmuebles_uso_propio', 'sum'),
       inversiones=('inversiones', 'sum'),
       patrimonio_neto=('patrimonio_neto', 'sum'),
   )

In [3]:
load_dotenv()
database_path = os.getenv('DATABASE')

In [4]:
with sqlite3.connect(database_path) as conn:
           base, parametros_reportes = get_recent_data(conn)

2025-01-28 22:06:40,832 - INFO - Base generada con 447,582 registros


In [5]:
parametros_reportes

,reporte,referencia,cod_cuenta,signo,id,concepto,es_subramo
0,ganaron-perdieron,1-rt,4.01.00.00.00.00.00.00,-1,6,resultado_tecnico,0
1,ganaron-perdieron,1-rt,5.01.00.00.00.00.00.00,1,6,resultado_tecnico,0
2,ganaron-perdieron,2-rf,5.02.00.00.00.00.00.00,1,7,resultado_financiero,0
3,ganaron-perdieron,2-rf,4.02.00.00.00.00.00.00,-1,7,resultado_financiero,0
4,ganaron-perdieron,3-roe,5.03.00.00.00.00.00.00,1,8,resultado_operaciones,0
5,ganaron-perdieron,3-roe,4.04.00.00.00.00.00.00,-1,8,resultado_operaciones,0
6,ganaron-perdieron,4-ig,4.05.00.00.00.00.00.00,1,9,impuesto_ganancias,0
7,nuevort,deud,2.01.01.00.00.00.00.00,1,10,deudas_con_asegurados,0
8,nuevort,deudreas,2.01.01.01.01.04.00.00,1,11,deudas_con_asegurados_ac_reaseguros,0
9,nuevort,disp,1.01.00.00.00.00.00.00,1,12,disponibilidades,0


In [10]:
ver = base[base['cod_cia']=='0381'].copy()

In [8]:
codigos_map = {
               concepto: dict(zip(
                   parametros_reportes[parametros_reportes['concepto'] == concepto]['cod_cuenta'],
                   parametros_reportes[parametros_reportes['concepto'] == concepto]['signo']
               ))
               for concepto in parametros_reportes['concepto'].unique()
           }

In [9]:
codigos_map

{'resultado_tecnico': {'4.01.00.00.00.00.00.00': -1,
  '5.01.00.00.00.00.00.00': 1},
 'resultado_financiero': {'5.02.00.00.00.00.00.00': 1,
  '4.02.00.00.00.00.00.00': -1},
 'resultado_operaciones': {'5.03.00.00.00.00.00.00': 1,
  '4.04.00.00.00.00.00.00': -1},
 'impuesto_ganancias': {'4.05.00.00.00.00.00.00': 1},
 'deudas_con_asegurados': {'2.01.01.00.00.00.00.00': 1},
 'deudas_con_asegurados_ac_reaseguros': {'2.01.01.01.01.04.00.00': 1},
 'disponibilidades': {'1.01.00.00.00.00.00.00': 1},
 'inmuebles_inversion': {'1.04.00.00.00.00.00.00': 1},
 'inmuebles_uso_propio': {'1.04.00.00.00.00.00.00': 1},
 'inversiones': {'1.02.00.00.00.00.00.00': 1},
 'patrimonio_neto': {'3.00.00.00.00.00.00.00': 1}}

In [11]:
generate_subramos_table(ver, codigos_map)

,cod_cia,periodo,resultado_tecnico,resultado_financiero,resultado_operaciones,impuesto_ganancias,deudas_con_asegurados,deudas_con_asegurados_ac_reaseguros,disponibilidades,inmuebles_inversion,inmuebles_uso_propio,inversiones,patrimonio_neto
0,0381,202401,574043030.0,425593696.0,0.0,255492658.0,406666923.0,-2045022.0,526103002.0,741600075.0,741600075.0,1.908619e+09,4.155659e+09
1,0381,202402,149605718.0,842623554.0,0.0,382973100.0,445848880.0,-8397847.0,307505792.0,876419852.0,876419852.0,2.329303e+09,4.654190e+09


In [12]:
generate_subramos_table(base, codigos_map)

,cod_cia,periodo,resultado_tecnico,resultado_financiero,resultado_operaciones,impuesto_ganancias,deudas_con_asegurados,deudas_con_asegurados_ac_reaseguros,disponibilidades,inmuebles_inversion,inmuebles_uso_propio,inversiones,patrimonio_neto
0,0002,202401,1.808516e+10,-1.595061e+10,0.0,-6.309666e+07,3.250936e+10,-2.039884e+10,4.259685e+09,7.328529e+09,7.328529e+09,4.441782e+10,2.708776e+10
1,0002,202402,1.998239e+10,-2.217290e+10,0.0,-3.641196e+09,4.123619e+10,-2.207767e+10,5.282901e+09,8.666145e+09,8.666145e+09,4.715780e+10,3.095882e+10
2,0002,202403,-1.032553e+09,-5.609406e+09,0.0,-2.613010e+09,4.619812e+10,-1.834625e+10,5.817366e+09,9.691613e+09,9.691613e+09,4.653963e+10,3.068549e+10
3,0006,202401,1.244753e+09,1.128672e+09,15910673.0,3.600000e+08,3.151980e+09,-2.585534e+06,4.802000e+07,1.039500e+09,1.039500e+09,7.453627e+09,5.261337e+09
4,0006,202402,6.701754e+09,5.789772e+08,18864719.0,1.810000e+09,3.776310e+09,-2.598404e+06,7.233000e+07,4.359095e+08,4.359095e+08,9.796253e+09,8.652694e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,0964,202402,3.391751e+09,-2.926239e+08,0.0,1.307875e+09,1.405302e+09,0.000000e+00,3.283486e+08,0.000000e+00,0.000000e+00,3.503172e+09,2.076410e+09
562,0964,202403,1.282806e+09,-9.672564e+06,0.0,4.949684e+08,1.709758e+09,0.000000e+00,1.506544e+09,0.000000e+00,0.000000e+00,5.049448e+09,3.106466e+09
563,0966,202401,6.154689e+07,-4.423785e+07,0.0,5.192714e+06,2.252883e+07,0.000000e+00,1.757776e+07,0.000000e+00,0.000000e+00,1.403345e+08,1.603542e+08
564,0966,202402,1.634041e+08,-1.060889e+08,0.0,3.222017e+07,2.880834e+07,0.000000e+00,3.385981e+06,0.000000e+00,0.000000e+00,1.671369e+08,2.008555e+08


In [ ]:
def main():
   load_dotenv()
   database_path = os.getenv('DATABASE')

   try:
       with sqlite3.connect(database_path) as conn:
           base, parametros_reportes = get_recent_data(conn)
           
           codigos_map = {
               concepto: dict(zip(
                   parametros_reportes[parametros_reportes['concepto'] == concepto]['cod_cuenta'],
                   parametros_reportes[parametros_reportes['concepto'] == concepto]['signo']
               ))
               for concepto in parametros_reportes['concepto'].unique()
           }

           result_df = generate_subramos_table(base, codigos_map)
           
           # Crear nueva tabla
           conn.execute("DROP TABLE IF EXISTS base_otros_conceptos")
           result_df.to_sql('base_otros_conceptos', conn, index=False)
           
           count = pd.read_sql_query("SELECT COUNT(*) as count FROM base_otros_conceptos", conn)
           logging.info(f"Tabla base_otros_conceptos creada con {count['count'].iloc[0]:,} registros")

   except sqlite3.Error as e:
       logging.error(f"Error en base de datos: {e}")

if __name__ == "__main__":
   main()